## IBM Data Science Professional Certificate
#### Module: Capstone, week 3, Project Title:
###  Clustering Toronto Neigbourhoods - part 3
##### author: Rafal Radecki
***

<p style="color:green;"><i>
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information.. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.
For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe so that it is in a structured format like the New York dataset.
Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.
</i></p>

### 7. Check the points on a map

In [1]:
# Load the geolocated neigbourhood data from teh saved csv file in part 2:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df3 = pd.read_csv('toronto-neighbourhoods-geo.csv')
df3.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.81139,-79.19662
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


In [2]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if packega is not installed
import folium # map rendering library

In [3]:
# Create a centre point coordinations for Toronto using the mean lat-lon from the dataframe:
tor_lat = df3['Latitude'].mean()
tor_lon = df3['Longitude'].mean()
print('Toronto mean coordinates /lat - lon/: ',tor_lat,' , ',tor_lon)

Toronto mean coordinates /lat - lon/:  43.70518519607846  ,  -79.39506941176467


<p style="color:green;">Visualise the points on interactive map using Folium package:</p>
<a href="https://python-visualization.github.io/folium/">python-visualization.github.io/folium</a> and <a href="https://leafletjs.com/reference-1.6.0.html#circlemarker">leafletjs.com</a> for formatting.

In [4]:
# create map of Toronto with points representing the neighbourhoods:
map_tor = folium.Map(location=[tor_lat, tor_lon], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        stroke=False,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

### 8. Find information for neigbourhood classification
<p style="color:green;">Now we can see loactions of the Neigbouhoods, it is time to evaluate them. For this we will use Foursquares and import information about venues available withing 1 km around the centre point. Then we'll attemp to cluster similar neighbourhoods</p>

Define Foursquare Credentials and Version

In [5]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [6]:
CLIENT_ID = 'JLWNZIP3YJZIGNUGTDRQFSLSPALSN1O4WACOLIZ0EO31NKPK' # your Foursquare ID
CLIENT_SECRET = 'VKDBGHHCCPCK0KI0NNE1D0KO0M2WWJ5SOPIQGR1C4F1CTDMB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JLWNZIP3YJZIGNUGTDRQFSLSPALSN1O4WACOLIZ0EO31NKPK
CLIENT_SECRET:VKDBGHHCCPCK0KI0NNE1D0KO0M2WWJ5SOPIQGR1C4F1CTDMB


In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # print(url)  
        
        # make the GET request: results = requests.get(url).json()["response"]['groups'][0]['items']
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
# code to run the above function on each neighborhood and create a new dataframe 
# called torronto_venues
toronto_venues = getNearbyVenues(names=df3['Neighbourhood'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [9]:
print(toronto_venues.shape)
toronto_venues.head()

(5020, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.81139,-79.19662,Canadiana exhibit,43.817962,-79.193374,Zoo Exhibit
1,"Rouge, Malvern",43.81139,-79.19662,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
2,"Rouge, Malvern",43.81139,-79.19662,Grizzly Bear Exhibit,43.817031,-79.193458,Zoo Exhibit
3,"Rouge, Malvern",43.81139,-79.19662,Upper Rouge Trail,43.809988,-79.186147,Trail
4,"Rouge, Malvern",43.81139,-79.19662,Lee Valley,43.803161,-79.199681,Hobby Shop


In [10]:
# save the results in a csv file:
# toronto_venues.to_csv(r'toronto-venues.csv', index=False)

### 9. Evaluate Neigbourhoods
<p style="color:green;">Now we have the information about venues in the neigbourhoods, its is time to clasify each of them based on the number of different venue types.</p>

In [12]:
#to avoid loading issues load toronto_venues df from a file obtained earlier:
toronto_venues = pd.read_csv('toronto-venues.csv')

# Group venues by neighbourhood and count them:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,44,44,44,44,44,44
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",26,26,26,26,26,26
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",14,14,14,14,14,14
"Alderwood, Long Branch",28,28,28,28,28,28
"Bathurst Manor, Downsview North, Wilson Heights",32,32,32,32,32,32
Bayview Village,7,7,7,7,7,7
"Bedford Park, Lawrence Manor East",36,36,36,36,36,36
Berczy Park,100,100,100,100,100,100


In [13]:
# find how many unique categories can be curated from all the returned venues:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category']
                                                    .unique())))

There are 331 uniques categories.


In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,ATM,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Pl

In [15]:
toronto_onehot.shape

(5057, 332)

<p style="color:green;">The 'one-hot encoding' created a df with the number of rows matching the number of venues obtained. Columns represents all venue category and '1' shows which category of venue the row represents.</p>
Next, let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category normalise the data set.

In [18]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,ATM,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Pl

In [19]:
toronto_grouped.shape

(102, 332)

Print each neighbourhood along with the top 10 most common venues:

In [20]:
num_top_venues = 10

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 3})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                    venue  freq
0             Coffee Shop  0.11
1                    Café  0.06
2                   Hotel  0.05
3                 Theater  0.04
4     Japanese Restaurant  0.03
5          Cosmetics Shop  0.03
6               Gastropub  0.03
7                Beer Bar  0.02
8                   Plaza  0.02
9  Furniture / Home Store  0.02


----Agincourt----
                venue   freq
0  Chinese Restaurant  0.182
1       Shopping Mall  0.114
2          Restaurant  0.045
3      Sandwich Place  0.045
4         Pizza Place  0.045
5      Clothing Store  0.023
6              Lounge  0.023
7        Noodle House  0.023
8                Park  0.023
9         Coffee Shop  0.023


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue   freq
0    Chinese Restaurant  0.154
1                  Park  0.115
2                Bakery  0.077
3              Pharmacy  0.077
4            Hobby Shop  0.038
5    Dim Sum Restaurant

To put the data in a dataframe we can first sort the venues:

In [21]:
# define the sorting function:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
import numpy as np # library to handle data in a vectorized manner

In [23]:
# Create the new dataframe and display the top 10 venues for each neighbourhood:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.shape
neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Theater,Cosmetics Shop,Gastropub,Japanese Restaurant,Restaurant,Beer Bar,Fast Food Restaurant
1,Agincourt,Chinese Restaurant,Shopping Mall,Sandwich Place,Pizza Place,Restaurant,Lounge,Coffee Shop,Bakery,Bank,Noodle House
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Park,Pharmacy,Bakery,Discount Store,Hobby Shop,Playground,Cantonese Restaurant,Shop & Service,BBQ Joint
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Caribbean Restaurant,Park,Sandwich Place,Home Service,Fried Chicken Joint,Pharmacy,Fast Food Restaurant,Auto Garage
4,"Alderwood, Long Branch",Coffee Shop,Pizza Place,Bank,Park,Print Shop,Greek Restaurant,Grocery Store,Gym,Italian Restaurant,Athletics & Sports


### 10. Cluster the neigbourhoods using K-Means model

In [24]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [25]:
# set number of clusters
kclusters = 9

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 0, 0, 6, 6, 6, 8, 5, 5, 5])

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood

# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.81139,-79.19662,7,Zoo Exhibit,Fast Food Restaurant,Trail,Hobby Shop,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.78574,-79.15875,1,Italian Restaurant,Burger Joint,Park,Breakfast Spot,Food,Flower Shop,Escape Room,Ethiopian Restaurant,Event Space,Food Court
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470,6,Convenience Store,Grocery Store,Park,Gymnastics Gym,Bus Line,Supermarket,Discount Store,Restaurant,Gym / Fitness Center,Athletics & Sports
3,M1G,Scarborough,Woburn,43.76812,-79.21761,0,Indian Restaurant,Pizza Place,Park,Fast Food Restaurant,Department Store,Discount Store,Supermarket,Sandwich Place,Juice Bar,Coffee Shop
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892,6,Indian Restaurant,Bakery,Coffee Shop,Bus Line,Music Store,Fried Chicken Joint,Caribbean Restaurant,Lounge,Gas Station,Supplement Shop


In [27]:
toronto_merged.tail()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,M9N,York,Weston,43.70507,-79.51804,6,Train Station,Intersection,Coffee Shop,Discount Store,Restaurant,Diner,Laundromat,Skating Rink,Park,Fried Chicken Joint
98,M9P,Etobicoke,Westmount,43.69630,-79.52926,6,Pizza Place,Gas Station,Breakfast Spot,Golf Driving Range,Pharmacy,Diner,Coffee Shop,Discount Store,Chinese Restaurant,Sandwich Place
99,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.68681,-79.55728,6,Pharmacy,Sandwich Place,Electronics Store,Coffee Shop,Bus Line,Pizza Place,Shopping Mall,Liquor Store,Supermarket,Beer Store
100,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.74453,-79.58624,6,Pizza Place,Grocery Store,Caribbean Restaurant,Park,Sandwich Place,Home Service,Fried Chicken Joint,Pharmacy,Fast Food Restaurant,Auto Garage
101,M9W,Etobicoke,Northwest,43.71174,-79.57941,6,Fast Food Restaurant,Bakery,Bank,Automotive Shop,Gas Station,Auto Dealership,Restaurant,Coffee Shop,Chinese Restaurant,Rental Car Location


In [28]:
# The Cluster Labels column is float type, convert it to int:
#toronto_merged['Cluster Labels'].astype(int)
#toronto_merged.head()
toronto_merged.to_csv(r'toronto-merged.csv', index=False)

Visualise clusters on the map

In [29]:
# create map of Toronto with clusters:
map_clusters = folium.Map(location=[tor_lat, tor_lon], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
markers_colors = []
for lat, lng, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        stroke=False,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)  
    
map_clusters

### 11. Examine the clusters

In [30]:
# Number of neighbourhoods per cluster:
print(toronto_merged['Cluster Labels'].value_counts())

5    46
6    43
0     5
8     2
3     2
7     1
4     1
2     1
1     1
Name: Cluster Labels, dtype: int64


Print neigbourhoods in some clusters:

In [31]:
# Cluster '1'
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",1,Italian Restaurant,Burger Joint,Park,Breakfast Spot,Food,Flower Shop,Escape Room,Ethiopian Restaurant,Event Space,Food Court


In [32]:
# Cluster '2'
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"Silver Hills, York Mills",2,Intersection,Gym,Gym / Fitness Center,Park,Farmers Market,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space


In [33]:
# Cluster '3'
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Roselawn,3,Pharmacy,Spa,Bank,Café,Trail,Sushi Restaurant,Skating Rink,Playground,Fish Market,Flower Shop
92,Islington Avenue,3,Pharmacy,Shopping Mall,Bank,Supermarket,Liquor Store,Grocery Store,Bus Stop,Skating Rink,Bakery,Café


In [34]:
# Cluster '4'
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Upper Rouge,4,Bakery,Zoo Exhibit,Field,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [35]:
# Cluster '7'
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",7,Zoo Exhibit,Fast Food Restaurant,Trail,Hobby Shop,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm


In [36]:
# Cluster '8'
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Bayview Village,8,Park,Bank,Chinese Restaurant,Café,Trail,Japanese Restaurant,Zoo Exhibit,Elementary School,Ethiopian Restaurant,Event Space
50,Rosedale,8,Park,Neighborhood,Trail,Playground,Sporting Goods Shop,Boutique,Grocery Store,Candy Store,Athletics & Sports,Zoo Exhibit


In [37]:
# Cluster '9'
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
